In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random





In [ ]:
IMAGE_SIZE = [224, 224]
random.seed(42)

In [ ]:
# initializing VGG with its own weights,
# include_top false means it removes top dense layers which are optimized for VGG images 
# we need to add new layers and train with our data.
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) # here [3] is 3 channels Red blue Green


In [ ]:
for layer in vgg.layers:
  layer.trainable = False
#Here we Froze the weights of VGG- Meaning we are using existing features from VGG adn mixing with our dense layers

In [ ]:
# Model without any dense layers, features only from VGNET and direct predictions.
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(26, activation='softmax')(x) # here we have 26 output classes so i gave 5 in output layer


In [ ]:
# Creating VGNET model
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:

model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)


In [ ]:
#Input needs to be (224,224)
# we are addign data augmentation to zoom in out , and flipping horizontally
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  #  shear_range = 0.2,
                                  #  zoom_range = 0.2,
                                  #  horizontal_flip = True
                                   )# add augments with horizontal flip

test_datagen = ImageDataGenerator(rescale = 1./255)#divide by 255 to normalize/scale

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Project_Data/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Project_Data/Test',
                                           target_size = (224, 224),
                                             batch_size = 128,
                                             class_mode = 'categorical')

Found 2366 images belonging to 26 classes.
Found 811 images belonging to 26 classes.


In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5
  # steps_per_epoch=128

)

Epoch 1/5
19/19 [==============================] - 61s 3s/step - loss: 6.8077 - accuracy: 0.0592 - val_loss: 3.6850 - val_accuracy: 0.1418
Epoch 2/5
19/19 [==============================] - 58s 3s/step - loss: 2.9673 - accuracy: 0.2443 - val_loss: 3.5062 - val_accuracy: 0.1566
Epoch 3/5
19/19 [==============================] - 59s 3s/step - loss: 3.0129 - accuracy: 0.2371 - val_loss: 3.5793 - val_accuracy: 0.1998
Epoch 4/5
19/19 [==============================] - 59s 3s/step - loss: 2.5977 - accuracy: 0.3233 - val_loss: 3.6798 - val_accuracy: 0.1973
Epoch 5/5
19/19 [==============================] - 57s 3s/step - loss: 2.0756 - accuracy: 0.4370 - val_loss: 3.2718 - val_accuracy: 0.2491


In [ ]:
#This did not work very will with VG net architecture.
#Transfer learning may not always work. we need to Tweak some parameters and try it agian,
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 15s 2s/step - loss: 3.2718 - accuracy: 0.2491
Test Loss: 3.271759033203125
Test accuracy: 0.24907521903514862


In [ ]:
#second try
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
x=Dense(100, activation='relu')(x)
Dropout(0.5)(x)
x=Dense(100, activation='relu')(x)
prediction = Dense(26, activation='softmax')(x)


In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  batch_size=128,
  callbacks=callback
)


Epoch 1/5
19/19 [==============================] - 61s 3s/step - loss: 3.8181 - accuracy: 0.0558 - val_loss: 3.2258 - val_accuracy: 0.0654
Epoch 2/5
19/19 [==============================] - 57s 3s/step - loss: 3.1838 - accuracy: 0.0875 - val_loss: 3.2157 - val_accuracy: 0.0592
Epoch 3/5
19/19 [==============================] - 57s 3s/step - loss: 3.1077 - accuracy: 0.1031 - val_loss: 3.1610 - val_accuracy: 0.0777
Epoch 4/5
19/19 [==============================] - 57s 3s/step - loss: 2.9927 - accuracy: 0.1230 - val_loss: 3.1100 - val_accuracy: 0.0838
Epoch 5/5
19/19 [==============================] - 57s 3s/step - loss: 2.9199 - accuracy: 0.1306 - val_loss: 3.0759 - val_accuracy: 0.1023


In [ ]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 15s 2s/step - loss: 3.0759 - accuracy: 0.1023
Test Loss: 3.0759243965148926
Test accuracy: 0.10234278440475464


In [ ]:
#Thrid Model - Now i will only use one dense layer and one for output.


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
x=Dense(50, activation='relu')(x)
prediction = Dense(26, activation='softmax')(x)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=8,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),callbacks=callback
)


Epoch 1/8
19/19 [==============================] - 60s 3s/step - loss: 2.6932 - accuracy: 0.1915 - val_loss: 2.8796 - val_accuracy: 0.1628
Epoch 2/8
19/19 [==============================] - 54s 3s/step - loss: 2.4677 - accuracy: 0.2756 - val_loss: 2.6992 - val_accuracy: 0.1961
Epoch 3/8
19/19 [==============================] - 51s 3s/step - loss: 2.2250 - accuracy: 0.3419 - val_loss: 2.5046 - val_accuracy: 0.2256
Epoch 4/8
19/19 [==============================] - 51s 3s/step - loss: 2.0159 - accuracy: 0.3918 - val_loss: 2.3632 - val_accuracy: 0.2676
Epoch 5/8
19/19 [==============================] - 52s 3s/step - loss: 1.7976 - accuracy: 0.4505 - val_loss: 2.2870 - val_accuracy: 0.2676
Epoch 6/8
19/19 [==============================] - 51s 3s/step - loss: 1.6061 - accuracy: 0.5072 - val_loss: 2.1680 - val_accuracy: 0.2922
Epoch 7/8
19/19 [==============================] - 51s 3s/step - loss: 1.4312 - accuracy: 0.5668 - val_loss: 2.1239 - val_accuracy: 0.3428
Epoch 8/8
19/19 [==========

In [ ]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 13s 2s/step - loss: 2.0350 - accuracy: 0.3613
Test Loss: 2.034989356994629
Test accuracy: 0.3612823784351349


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]

In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=8,
  # steps_per_epoch=len(training_set),
  # validation_steps=len(test_set)
  callbacks=callback
)

Epoch 1/8
19/19 [==============================] - 74s 3s/step - loss: 2.4126 - accuracy: 0.4159 - val_loss: 2.1998 - val_accuracy: 0.2996
Epoch 2/8
19/19 [==============================] - 51s 3s/step - loss: 1.5023 - accuracy: 0.5237 - val_loss: 2.4189 - val_accuracy: 0.2651
Epoch 3/8
19/19 [==============================] - 51s 3s/step - loss: 1.5190 - accuracy: 0.5304 - val_loss: 2.5335 - val_accuracy: 0.2589
Epoch 4/8
19/19 [==============================] - 50s 3s/step - loss: 1.4429 - accuracy: 0.5359 - val_loss: 2.5023 - val_accuracy: 0.2774
Epoch 5/8
19/19 [==============================] - 51s 3s/step - loss: 1.4192 - accuracy: 0.5385 - val_loss: 2.2977 - val_accuracy: 0.2972
Epoch 6/8
19/19 [==============================] - 51s 3s/step - loss: 1.3890 - accuracy: 0.5469 - val_loss: 2.2306 - val_accuracy: 0.3292
Epoch 00006: early stopping


In [ ]:
#This model Fitted Best compared to the model whit direct predictions of our data with VGG 
#it worked better than the one with 2 dense layers.

In [ ]:
# Its good to use simple models than complex models with more features.

In [ ]:
# Here we explored the Power of transfer learning, by using existing features from VGG
# and adding only one dense layer other than output we saved a lot of compute power and time and
#achieved great accuracy.

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:

last_layer = vgg.get_layer('block4_pool')

print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

x = Flatten()(last_output)

# x = Dense(1000, activation='relu')(x)
# x=Dense(100, activation='relu')(x)
# Dropout(0.5)(x)
x=Dense(100, activation='relu')(x)
Dropout(0.5)(x)
prediction = Dense(26, activation='softmax')(x)


last layer output shape:  (None, 14, 14, 512)


In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  batch_size=128,
  callbacks=callback
)


Epoch 1/5
19/19 [==============================] - 53s 3s/step - loss: 19.9695 - accuracy: 0.0545 - val_loss: 3.2572 - val_accuracy: 0.0382
Epoch 2/5
19/19 [==============================] - 50s 3s/step - loss: 3.3698 - accuracy: 0.0600 - val_loss: 3.2838 - val_accuracy: 0.0407
Epoch 3/5
19/19 [==============================] - 50s 3s/step - loss: 3.2715 - accuracy: 0.0630 - val_loss: 3.2626 - val_accuracy: 0.0382
Epoch 4/5
19/19 [==============================] - 50s 3s/step - loss: 3.2784 - accuracy: 0.0558 - val_loss: 3.2586 - val_accuracy: 0.0382
Epoch 5/5
19/19 [==============================] - 50s 3s/step - loss: 3.2443 - accuracy: 0.0587 - val_loss: 3.2611 - val_accuracy: 0.0395


In [ ]:
# we can see that Image net is not a great fit for our dataset, even after training and removing some layers.